In [325]:
import requests
import json


In [326]:
evnapi_server = 'hw-en-pcrs-ap01'
evnapi_port = '8088'
evn_api_endpoint = '/v1/evn/assets/'

asset = 'abn3113'


In [327]:
def get_page(asset_id):
    url = 'http://'+evnapi_server+':'+evnapi_port+evn_api_endpoint+asset_id
    r = requests.get(url)
    if (not r.ok):
        exit('Failed to load asset '+asset_id)
    return json.loads(r.content)

In [364]:
def drop_unnecessary(metadata):
    remove = ['_id', 'content_asset_id', 'id', 'last_updated', 'codewords', 'asset_relations', 'facets', 'behaviorChange', 'codes', 'keywords', 'emr_max_age', 'emr_min_age','specialties','cached', 'hash']
    for remove_me in remove:
        del metadata[remove_me]

In [385]:
def concepts_to_int(metadata):
    # Handle nested concepts    
    concept_ids = set()
    for concept in metadata['concepts']:
        int_id = int(concept['concept_id'][5:])
        concept_ids.add(int_id)
    return list(concept_ids)

In [386]:
def categories_to_int(metadata):
    category_ids = set()
    for cat in metadata['categories']:
        c = int(cat['id'][3:])
        category_ids.add(c)
    return list(category_ids)

In [387]:
def genders_to_int(metadata):
    
    # 0 = female, 1 = male, 2 = both
    
    gender = 0
    if (len(metadata['gender']) > 1):
        gender = 2
    elif (metadata['gender'][0] == 'male'):
        gender = 1
    else:
        gender = 0
    return gender

In [388]:
def audience_to_int(metadata):
    
    # 0 = none, 1 = Caregiver, 2 = Parent, 3 = Patient, 4 = many
    
   
    if (len(metadata['audience']) == 0):
        return 0
    elif (len(metadata['audience']) > 1):
        return 4
    elif (metadata['audience'][0] == 'Caregiver'):
        return 1
    elif (metadata['audience'][0] == 'Parent'):
        return 2
    else:
        return 3


In [389]:
def settings_to_int(metadata):
    
    # 0 = none, inpatient = 1, outpatient = 2, many = 3
    
 
    if (len(metadata['delivery_setting']) == 0):
        return 0
    elif (len(metadata['delivery_setting']) > 1):
        return 3
    elif (metadata['delivery_setting'][0] == 'Outpatient'):
        return 2
    else:
        return 1
    return setting

In [402]:
def clean_metadata(metadata):
    drop_unnecessary(metadata)
        
    # Handle nested concepts    
    concept_ids = concepts_to_int(metadata)
    metadata['concepts'] = concept_ids
    
    
    # Handle nested categories
    category_ids = categories_to_int(metadata)

    metadata['categories'] = category_ids
    
    # Handle Gender
    genders = genders_to_int(metadata)
    metadata['gender'] = genders
    
    # Handle Gender
    settings = setting_to_int(metadata)
    del metadata['delivery_setting']
    metadata['setting'] = settings
    
    #Handle Audience
    audience = audience_to_int(metadata)
    metadata['audience'] = audience
    

    
    
    return metadata

In [403]:
response = get_page(asset)

In [404]:
x = clean_metadata(response)

In [405]:
x

{'audience': 2,
 'categories': [1028, 1007],
 'concepts': [5064],
 'gender': 2,
 'max_age': 18,
 'min_age': 2,
 'setting': 1}